# Faza 2 i 3: Od Notebooka do Zautomatyzowanego Potoku MLOps

Ten notatnik stanowi praktyczny przewodnik po procesie przekształcania eksperymentalnego kodu z notatnika Jupyter w modularny, reużywalny i zautomatyzowany potok uczenia maszynowego przy użyciu **Kubeflow Pipelines (KFP)** i **Vertex AI Pipelines**.

Celem jest demonstracja, jak logika z prototypu (Faza 1) zostaje podzielona na niezależne **komponenty**, a następnie połączona w spójny graf wykonania (**pipeline**), który zarządza całym cyklem życia modelu – od danych surowych po wdrożenie.

### Kluczowe korzyści tego podejścia:

* **Powtarzalność:** Każde uruchomienie potoku wykonuje te same kroki w identyczny sposób.
* **Reużywalność:** Komponenty mogą być wykorzystywane w innych projektach.
* **Skalowalność:** Poszczególne kroki mogą być uruchamiane na zasobach chmurowych o odpowiedniej mocy obliczeniowej.
* **Łatwość w utrzymaniu:** Zmiany w jednym komponencie nie wpływają na inne, o ile interfejsy (wejścia/wyjścia) pozostają te same.
* **Automatyzacja:** Potok stanowi podstawę dla systemów CI/CD, umożliwiając automatyczne trenowanie i wdrażanie modeli.

## 1. Konfiguracja środowiska i import bibliotek

Na początku importujemy niezbędne biblioteki, w tym KFP, oraz definiujemy zmienne konfiguracyjne dla naszego projektu w Google Cloud.

In [ ]:
import kfp
from kfp import dsl
from kfp.dsl import Input, Output, Dataset, Model, Metrics

# --- Zmienne konfiguracyjne ---
# Uzupełnij poniższe zmienne swoimi wartościami z Google Cloud
PROJECT_ID = "twoj-google-cloud-project-id"  # Wstaw swój Project ID
BUCKET_NAME = "gs://twoj-gcs-bucket-name"     # Wstaw nazwę swojego bucketu w GCS
PIPELINE_ROOT = f"{BUCKET_NAME}/pipeline-root" # Ścieżka do przechowywania artefaktów potoku
REGION = "europe-central2"                   # Region, w którym działa Vertex AI, np. europe-central2 dla Warszawy

## 2. Tworzenie Komponentów Potoku

**Komponent KFP** to samodzielna część potoku, która realizuje jedno, konkretne zadanie. Jest to funkcja w języku Python, opatrzona dekoratorem `@dsl.component`. Dekorator ten zawiera metadane, takie jak obraz bazowy kontenera Docker oraz pakiety do zainstalowania. Dzięki temu każdy komponent jest uruchamiany w izolowanym, powtarzalnym środowisku.

Poniżej zdefiniujemy komponenty dla każdego kroku naszego procesu ML.

### Komponent 1: Pobieranie i ładowanie danych

Ten komponent jest odpowiedzialny za pobranie surowego zbioru danych z publicznego zasobu Google Cloud Storage i przekazanie go jako artefakt typu `Dataset` do kolejnego kroku w potoku.

In [ ]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["pandas", "fsspec", "gcsfs", "scikit-learn"]
)
def load_data(
    dataset: Output[Dataset]
):
    """Pobiera dane o pingwinach i zapisuje je jako artefakt."""
    import pandas as pd
    
    csv_url = "gs://cloud-samples-data/vertex-ai/pipeline-introduction/penguins.csv"
    df = pd.read_csv(csv_url)
    
    df.to_csv(dataset.path, index=False)
    print(f"Data loaded and saved to {dataset.path}")

### Komponent 2: Przygotowanie i podział danych

Komponent ten przyjmuje surowe dane, wykonuje na nich operacje czyszczenia i inżynierii cech (zgodnie z logiką z notatnika), a następnie dzieli je na zbiory treningowe i testowe. Każdy z tych podzbiorów jest zapisywany jako osobny artefakt wyjściowy.

In [ ]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["pandas", "scikit-learn"]
)
def preprocess_data(
    dataset_in: Input[Dataset],
    x_train: Output[Dataset],
    x_test: Output[Dataset],
    y_train: Output[Dataset],
    y_test: Output[Dataset]
):
    """Czyści dane, dokonuje inżynierii cech i dzieli zbiór na treningowy i testowy."""
    import pandas as pd
    from sklearn.model_selection import train_test_split

    df = pd.read_csv(dataset_in.path)

    # Czyszczenie danych (zgodnie z notatnikiem)
    df.loc[336, 'sex'] = 'FEMALE' # Poprawienie błędnej wartości
    df_clean = df.copy()
    numerical_cols = ['culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g']
    for col in numerical_cols:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())
    df_clean['sex'] = df_clean['sex'].fillna(df_clean['sex'].mode()[0])
    
    # Inżynieria cech
    df_clean['sex'] = df_clean['sex'].map({'MALE': 0, 'FEMALE': 1})
    df_processed = pd.get_dummies(df_clean, columns=['island'], drop_first=True)

    # Zdefiniowanie cech (X) i etykiety (y)
    X = df_processed.drop('species', axis=1)
    y = df_processed['species']

    # Podział na zbiór treningowy i testowy
    X_train_df, X_test_df, y_train_ser, y_test_ser = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Zapisanie podzielonych danych jako artefakty
    X_train_df.to_csv(x_train.path, index=False)
    X_test_df.to_csv(x_test.path, index=False)
    y_train_ser.to_csv(y_train.path, index=False, header=False)
    y_test_ser.to_csv(y_test.path, index=False, header=False)
    print("Data preprocessed and split successfully.")

### Komponent 3: Trenowanie modelu

Ten komponent odpowiada za proces trenowania. Przyjmuje dane treningowe i tworzy potok `scikit-learn` zawierający skaler i klasyfikator SVC. Wytrenowany model jest następnie zapisywany jako artefakt typu `Model`, co pozwala na jego wersjonowanie i śledzenie w Vertex AI.

In [ ]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["pandas", "scikit-learn", "joblib"]
)
def train_model(
    x_train_in: Input[Dataset],
    y_train_in: Input[Dataset],
    model_out: Output[Model]
):
    """Trenuje model SVC przy użyciu potoku scikit-learn."""
    import pandas as pd
    import joblib
    from sklearn.preprocessing import StandardScaler
    from sklearn.svm import SVC
    from sklearn.pipeline import Pipeline

    X_train = pd.read_csv(x_train_in.path)
    y_train = pd.read_csv(y_train_in.path, header=None).squeeze()

    # Definicja potoku scikit-learn
    svc_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(kernel='linear', probability=True, random_state=42))
    ])

    # Trenowanie modelu
    svc_pipeline.fit(X_train, y_train)

    # Zapisanie modelu jako artefakt
    model_out.metadata["framework"] = "scikit-learn"
    model_out.metadata["containerSpec"] = {
        "imageUri": "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
    }
    joblib.dump(svc_pipeline, model_out.path)
    print(f"Model trained and saved to {model_out.path}")

### Komponent 4: Ocena modelu

Po wytrenowaniu model musi zostać oceniony. Ten komponent wczytuje model oraz dane testowe, dokonuje predykcji i oblicza metrykę dokładności (*accuracy*). Wynik jest zapisywany jako artefakt typu `Metrics`, co umożliwia jego wizualizację w interfejsie Vertex AI Pipelines i wykorzystanie w krokach warunkowych.

In [ ]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["pandas", "scikit-learn", "joblib"]
)
def evaluate_model(
    x_test_in: Input[Dataset],
    y_test_in: Input[Dataset],
    model_in: Input[Model],
    metrics_out: Output[Metrics]
):
    """Ocenia model na zbiorze testowym i zapisuje metryki."""
    import pandas as pd
    import joblib
    from sklearn.metrics import accuracy_score

    X_test = pd.read_csv(x_test_in.path)
    y_test = pd.read_csv(y_test_in.path, header=None).squeeze()
    
    model = joblib.load(model_in.path)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Zapisanie metryk jako artefakt
    metrics_out.log_metric("accuracy", accuracy)
    print(f"Model accuracy: {accuracy}")

### Komponent 5: Warunkowe wdrożenie modelu

Ostatni komponent odpowiada za proces ciągłego dostarczania (CD). Jego zadaniem jest wdrożenie modelu na **Vertex AI Endpoint**, ale tylko wtedy, gdy spełniony jest określony warunek – w tym przypadku, gdy dokładność modelu na zbiorze testowym przekracza zdefiniowany próg. Komponent ten wykorzystuje SDK `google-cloud-aiplatform` do interakcji z usługami Google Cloud.

In [ ]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["google-cloud-aiplatform"]
)
def deploy_model_to_vertex(
    project: str,
    location: str,
    model_in: Input[Model],
    metrics_in: Input[Metrics],
    min_accuracy: float,
    endpoint_out: Output[dsl.Artifact]
):
    """Wdraża model na Vertex AI Endpoint, jeśli dokładność jest wystarczająca."""
    from google.cloud import aiplatform
    
    accuracy = metrics_in.metadata["accuracy"]
    print(f"Current model accuracy: {accuracy}")

    if accuracy >= min_accuracy:
        print(f"Accuracy {accuracy} >= {min_accuracy}. Deploying model.")
        
        aiplatform.init(project=project, location=location)

        # Wgranie modelu do Vertex AI Model Registry
        uploaded_model = aiplatform.Model.upload(
            display_name="penguin-classifier-pipeline",
            artifact_uri=model_in.uri.replace("/model", ""), # Wymagany jest URI do katalogu
            serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
        )
        
        # Stworzenie i wdrożenie punktu końcowego (endpoint)
        endpoint = uploaded_model.deploy(
            machine_type="n1-standard-2",
            min_replica_count=1,
            max_replica_count=1,
            traffic_split={"0": 100},
            sync=True
        )
        
        endpoint_out.uri = endpoint.resource_name
        print(f"Model deployed to endpoint: {endpoint.resource_name}")
    else:
        print(f"Accuracy {accuracy} < {min_accuracy}. Skipping deployment.")

## 3. Definicja i Kompilacja Potoku

Mając zdefiniowane wszystkie komponenty, możemy teraz połączyć je w jeden, spójny potok. Funkcja oznaczona dekoratorem `@dsl.pipeline` definiuje graf wykonania (DAG), określając, które komponenty zależą od wyników (artefaktów) innych.

W naszym przypadku:
1.  `load_data` uruchamia się jako pierwszy.
2.  `preprocess_data` czeka na dane z `load_data`.
3.  `train_model` czeka na dane treningowe z `preprocess_data`.
4.  `evaluate_model` czeka na model z `train_task` i dane testowe z `preprocess_data`.
5.  `deploy_model_to_vertex` jest uruchamiany warunkowo, na podstawie metryki z `evaluate_model`.

In [ ]:
@dsl.pipeline(
    name="penguin-classification-pipeline",
    description="Zautomatyzowany potok ML do klasyfikacji pingwinów.",
    pipeline_root=PIPELINE_ROOT,
)
def penguin_classification_pipeline(
    min_accuracy_threshold: float = 0.95,
    project_id: str = PROJECT_ID,
    region: str = REGION
):
    """Orkiestruje przepływ pracy od danych do wdrożonego modelu."""
    
    load_task = load_data()

    preprocess_task = preprocess_data(
        dataset_in=load_task.outputs["dataset"]
    )

    train_task = train_model(
        x_train_in=preprocess_task.outputs["x_train"],
        y_train_in=preprocess_task.outputs["y_train"]
    )

    evaluate_task = evaluate_model(
        x_test_in=preprocess_task.outputs["x_test"],
        y_test_in=preprocess_task.outputs["y_test"],
        model_in=train_task.outputs["model_out"]
    )

    # Krok warunkowy
    with dsl.Condition(
        evaluate_task.outputs["metrics_out"].metadata["accuracy"] >= min_accuracy_threshold,
        name="deploy-condition",
    ):
        deploy_task = deploy_model_to_vertex(
            project=project_id,
            location=region,
            model_in=train_task.outputs["model_out"],
            metrics_in=evaluate_task.outputs["metrics_out"],
            min_accuracy=min_accuracy_threshold
        )

## 4. Kompilacja i Uruchomienie

Ostatnim krokiem jest skompilowanie definicji potoku do pliku w formacie JSON. Ten plik zawiera wszystkie informacje potrzebne usłudze Vertex AI Pipelines do wykonania naszego potoku. 

Po skompilowaniu, plik `penguin_pipeline.json` można wgrać do interfejsu Vertex AI, aby uruchomić zadanie, lub zrobić to programistycznie za pomocą SDK, jak pokazano w zakomentowanym kodzie poniżej.

In [ ]:
if __name__ == '__main__':
    # Kompilacja potoku do pliku JSON
    kfp.compiler.Compiler().compile(
        pipeline_func=penguin_classification_pipeline,
        package_path="penguin_pipeline.json"
    )
    print("Pipeline compiled to penguin_pipeline.json")

    # Uruchomienie potoku w Vertex AI (wymaga uwierzytelnienia w gcloud)
    # from google.cloud import aiplatform
    # 
    # aiplatform.init(project=PROJECT_ID, location=REGION)
    # 
    # job = aiplatform.PipelineJob(
    #     display_name="penguin-pipeline-run",
    #     template_path="penguin_pipeline.json",
    #     pipeline_root=PIPELINE_ROOT,
    #     enable_caching=True
    # )
    # 
    # job.run()
    # print("Pipeline job started.")